# Hosting Semantic Kernel Agents with Amazon Bedrock models in Amazon Bedrock AgentCore Runtime

## Overview

In this tutorial we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. 

We will focus on a Semantic Kernel agent with Amazon Bedrock model example. For other framework examples, check:
- [Strands Agents with Amazon Bedrock model](../01-strands-with-bedrock-model)
- [LangGraph with Amazon Bedrock model](../02-langgraph-with-bedrock-model)
- [Strands Agents with OpenAI model](../03-strands-with-openai-model)
- [CrewAI with Amazon Bedrock model](../04-crewai-with-bedrock-model)

### Tutorial Details

| Information         | Details                                                                               |
|:--------------------|:--------------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                        |
| Agent type          | Single                                                                                |
| Agentic Framework   | Semantic Kernel                                                                       |
| LLM model           | Anthropic Claude Sonnet 3                                                             |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Semantic Kernel and Amazon Bedrock Model    |
| Tutorial vertical   | Cross-vertical                                                                        |
| Example complexity  | Easy                                                                                  |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                          |

### Tutorial Architecture

In this tutorial we will describe how to deploy an existing agent to AgentCore runtime. 

For demonstration purposes, we will use a Semantic Kernel agent using Amazon Bedrock models

In our example we will use a very simple agent with three tools: `calculator`, `weather`, and `get_time`. 

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Amazon Bedrock models
* Using Microsoft Semantic Kernel framework
* Function calling with custom plugins

## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Semantic Kernel
* Docker running

In [1]:
#!uv add -r requirements.txt --active

In [2]:
print("hello")

hello


## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

The architecture here will look as following:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="50%"/>
</div>

In [3]:
%%writefile semantic_kernel_bedrock.py
import asyncio
import json
import math
import operator
from datetime import datetime
from typing import Annotated
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.bedrock.services.bedrock_chat_completion import BedrockChatCompletion
from semantic_kernel.connectors.ai.bedrock.bedrock_prompt_execution_settings import BedrockChatPromptExecutionSettings
from semantic_kernel.functions import kernel_function
from semantic_kernel.contents import ChatHistory
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
import argparse

class UtilsPlugin:
    """A plugin providing calculator and weather tools"""
    
    @kernel_function(
        name="calculator",
        description="Calculate the result of a mathematical expression"
    )
    def calculator(
        self,
        expression: Annotated[str, "A mathematical expression like '2+2' or 'sqrt(16)'"]
    ) -> str:
        """Calculate the result of a mathematical expression."""
        try:
            # Define safe functions that can be used in expressions
            safe_dict = {
                "__builtins__": {},
                "abs": abs, "round": round, "min": min, "max": max,
                "sum": sum, "pow": pow,
                # Math functions
                "sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan,
                "log": math.log, "log10": math.log10, "exp": math.exp,
                "pi": math.pi, "e": math.e,
                "ceil": math.ceil, "floor": math.floor,
                "degrees": math.degrees, "radians": math.radians,
            }
            
            # Evaluate the expression safely
            result = eval(expression, safe_dict)
            return str(result)
            
        except Exception as e:
            return f"Error: {str(e)}"
    
    @kernel_function(
        name="weather",
        description="Get the current weather (dummy implementation)"
    )
    def weather(self) -> str:
        """Get weather - dummy implementation that always returns sunny"""
        return "sunny"
    
    @kernel_function(
        name="get_time",
        description="Get the current time"
    )
    def get_time(self) -> str:
        """Get the current time"""
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Create and configure the agent
def create_agent():
    """Create and configure the Semantic Kernel agent"""
    # Initialize the kernel
    kernel = Kernel()
    
    # Add Bedrock as the chat completion service
    chat_service = BedrockChatCompletion(
        model_id="us.anthropic.claude-3-sonnet-20240229-v1:0"
    )
    kernel.add_service(chat_service)
    
    # Add the utilities plugin
    kernel.add_plugin(UtilsPlugin(), plugin_name="Utils")
    
    return kernel

# Initialize the agent
kernel = create_agent()

def semantic_kernel_bedrock(payload):
    """Invoke the agent with a payload"""
    user_input = payload.get("prompt")
    return asyncio.run(invoke_agent(user_input))

async def invoke_agent(user_input: str) -> str:
    """Async function to invoke the agent"""
    # Get the chat completion service
    chat_completion = kernel.get_service(type=ChatCompletionClientBase)
    
    # Configure execution settings with function calling
    execution_settings = BedrockChatPromptExecutionSettings()
    execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
    
    # Create chat history
    history = ChatHistory()
    history.add_system_message(
        "You're a helpful assistant. You can do simple math calculations, "
        "tell the weather, and get the current time."
    )
    history.add_user_message(user_input)
    
    # Get response from the model
    response = await chat_completion.get_chat_message_content(
        chat_history=history,
        settings=execution_settings,
        kernel=kernel,
    )
    
    return response.content

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = semantic_kernel_bedrock(json.loads(args.payload))
    print(response)

Overwriting semantic_kernel_bedrock.py


#### Invoking local agent

In [4]:
!python semantic_kernel_bedrock.py '{"prompt": "What is the weather now?"}'

The weather tool is just providing a dummy response, but it indicates sunny weather right now.


In [5]:
!python semantic_kernel_bedrock.py '{"prompt": "What is 15 * 23?"}'

The result of 15 * 23 is 345.


In [6]:
!python semantic_kernel_bedrock.py '{"prompt": "What time is it?"}'

The current time is 12:12 AM.


## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### Semantic Kernel with Amazon Bedrock model
Let's update our Semantic Kernel agent to work with AgentCore Runtime:

In [7]:
%%writefile semantic_kernel_bedrock.py
import asyncio
import json
import math
import operator
from datetime import datetime
from typing import Annotated
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.bedrock.services.bedrock_chat_completion import BedrockChatCompletion
from semantic_kernel.connectors.ai.bedrock.bedrock_prompt_execution_settings import BedrockChatPromptExecutionSettings
from semantic_kernel.functions import kernel_function
from semantic_kernel.contents import ChatHistory
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

class UtilsPlugin:
    """A plugin providing calculator and weather tools"""
    
    @kernel_function(
        name="calculator",
        description="Calculate the result of a mathematical expression"
    )
    def calculator(
        self,
        expression: Annotated[str, "A mathematical expression like '2+2' or 'sqrt(16)'"]
    ) -> str:
        """Calculate the result of a mathematical expression."""
        try:
            # Define safe functions that can be used in expressions
            safe_dict = {
                "__builtins__": {},
                "abs": abs, "round": round, "min": min, "max": max,
                "sum": sum, "pow": pow,
                # Math functions
                "sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan,
                "log": math.log, "log10": math.log10, "exp": math.exp,
                "pi": math.pi, "e": math.e,
                "ceil": math.ceil, "floor": math.floor,
                "degrees": math.degrees, "radians": math.radians,
            }
            
            # Evaluate the expression safely
            result = eval(expression, safe_dict)
            return str(result)
            
        except Exception as e:
            return f"Error: {str(e)}"
    
    @kernel_function(
        name="weather",
        description="Get the current weather (dummy implementation)"
    )
    def weather(self) -> str:
        """Get weather - dummy implementation that always returns sunny"""
        return "sunny"
    
    @kernel_function(
        name="get_time",
        description="Get the current time"
    )
    def get_time(self) -> str:
        """Get the current time"""
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Create and configure the agent
def create_agent():
    """Create and configure the Semantic Kernel agent"""
    # Initialize the kernel
    kernel = Kernel()
    
    # Add Bedrock as the chat completion service
    chat_service = BedrockChatCompletion(
        model_id="us.anthropic.claude-3-sonnet-20240229-v1:0"
    )
    kernel.add_service(chat_service)
    
    # Add the utilities plugin
    kernel.add_plugin(UtilsPlugin(), plugin_name="Utils")
    
    return kernel

# Initialize the agent
kernel = create_agent()

@app.entrypoint
def semantic_kernel_bedrock(payload):
    """Invoke the agent with a payload"""
    user_input = payload.get("prompt")
    print("User input:", user_input)
    return asyncio.run(invoke_agent(user_input))

async def invoke_agent(user_input: str) -> str:
    """Async function to invoke the agent"""
    # Get the chat completion service
    chat_completion = kernel.get_service(type=ChatCompletionClientBase)
    
    # Configure execution settings with function calling
    execution_settings = BedrockChatPromptExecutionSettings()
    execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
    
    # Create chat history
    history = ChatHistory()
    history.add_system_message(
        "You're a helpful assistant. You can do simple math calculations, "
        "tell the weather, and get the current time."
    )
    history.add_user_message(user_input)
    
    # Get response from the model
    response = await chat_completion.get_chat_message_content(
        chat_history=history,
        settings=execution_settings,
        kernel=kernel,
    )
    
    return response.content

if __name__ == "__main__":
    app.run()

Overwriting semantic_kernel_bedrock.py


## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial we will use the Amazon Bedrock AgentCore Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Creating runtime role

Before starting, let's create an IAM role for our AgentCore Runtime. We will do so using the utils function pre-developed for you.

In [8]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.join(utils_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import create_agentcore_role

agent_name="agentcore_semantic_kernel"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

sys.path[0]: /Users/claumazz/PycharmProjects/amazon-bedrock-agentcore-samples/01-tutorials
attaching role policy agentcore-agentcore_semantic_kernel-role


### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [14]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="semantic_kernel_bedrock.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

Bedrock AgentCore configured: /Users/claumazz/PycharmProjects/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/05-semantic-kernel-with-bedrock-model/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/Users/claumazz/PycharmProjects/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/05-semantic-kernel-with-bedrock-model/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/claumazz/PycharmProjects/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/05-semantic-kernel-with-bedrock-model/Dockerfile'), dockerignore_path=PosixPath('/Users/claumazz/PycharmProjects/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/05-semantic-kernel-with-bedrock-model/.dockerignore'), runtime='Docker', region='us-east-1', account_id='686255965275', execution_role='arn:aws:iam::686255965275:role/agentcore-agentcore_semantic_kernel-role', ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [15]:
launch_result = agentcore_runtime.launch()

The push refers to repository [686255965275.dkr.ecr.us-east-1.amazonaws.com/bedrock_agentcore-agentcore_semantic_kernel]
27b1542b9257: Waiting
84bb154987a1: Waiting
b3407f3b5b5b: Waiting
c7ced59943c6: Waiting
b5b8c27c1e67: Waiting
046e0ba021a5: Waiting
64bd21e63ff7: Waiting
94fa42b5194c: Waiting
823b52aff6ed: Waiting
b4270a851ef2: Waiting
84bb154987a1: Waiting
b3407f3b5b5b: Waiting
c7ced59943c6: Waiting
b5b8c27c1e67: Waiting
27b1542b9257: Waiting
64bd21e63ff7: Waiting
94fa42b5194c: Waiting
823b52aff6ed: Waiting
b4270a851ef2: Waiting
046e0ba021a5: Waiting
64bd21e63ff7: Waiting
94fa42b5194c: Waiting
823b52aff6ed: Waiting
b4270a851ef2: Waiting
046e0ba021a5: Waiting
84bb154987a1: Waiting
b3407f3b5b5b: Waiting
c7ced59943c6: Waiting
b5b8c27c1e67: Waiting
27b1542b9257: Waiting
84bb154987a1: Pushed
64bd21e63ff7: Pushed
94fa42b5194c: Pushed
823b52aff6ed: Pushed
b4270a851ef2: Pushed
b5b8c27c1e67: Pushed
27b1542b9257: Pushed
046e0ba021a5: Pushed
b3407f3b5b5b: Pushed
c7ced59943c6: Pushed
latest: d

Deployed to cloud: arn:aws:bedrock-agentcore:us-east-1:686255965275:runtime/agentcore_semantic_kernel-znCs468wCI
🔍 Agent logs available at:
   /aws/bedrock-agentcore/runtimes/agentcore_semantic_kernel-znCs468wCI-DEFAULT
   /aws/bedrock-agentcore/runtimes/agentcore_semantic_kernel-znCs468wCI-DEFAULT/runtime-logs
💡 Tail logs with: aws logs tail /aws/bedrock-agentcore/runtimes/agentcore_semantic_kernel-znCs468wCI-DEFAULT --follow
💡 Or view recent logs: aws logs tail /aws/bedrock-agentcore/runtimes/agentcore_semantic_kernel-znCs468wCI-DEFAULT --since 1h


### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [16]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: agentcore_semantic_kernel


'READY'

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [17]:
invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now?"})
invoke_response

{'ResponseMetadata': {'RequestId': '2486eb1a-63e7-46e3-a19d-1d94180670ad',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 05 Aug 2025 04:23:22 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2486eb1a-63e7-46e3-a19d-1d94180670ad',
   'baggage': 'Self=1-68918737-1109137958536fa23b6f3336,session.id=7ee2ed9f-b69b-4fc8-b0cc-7661d565a94c',
   'x-amzn-bedrock-agentcore-runtime-session-id': '7ee2ed9f-b69b-4fc8-b0cc-7661d565a94c',
   'x-amzn-trace-id': 'Root=1-68918737-66367e66351a06965da7dd00;Self=1-68918737-1109137958536fa23b6f3336'},
  'RetryAttempts': 0},
 'runtimeSessionId': '7ee2ed9f-b69b-4fc8-b0cc-7661d565a94c',
 'traceId': 'Root=1-68918737-66367e66351a06965da7dd00;Self=1-68918737-1109137958536fa23b6f3336',
 'baggage': 'Self=1-68918737-1109137958536fa23b6f3336,session.id=7ee2ed9f-b69b-4fc8-b0cc-7661d565a94c',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': [b'"The current

In [18]:
invoke_response = agentcore_runtime.invoke({"prompt": "What is sqrt(144)?"})
invoke_response

{'ResponseMetadata': {'RequestId': 'e5c0b93b-ceef-460b-8ee4-d00c859bef28',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 05 Aug 2025 04:26:32 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e5c0b93b-ceef-460b-8ee4-d00c859bef28',
   'baggage': 'Self=1-689187f4-7960c6cc6b4e66233d854361,session.id=7ee2ed9f-b69b-4fc8-b0cc-7661d565a94c',
   'x-amzn-bedrock-agentcore-runtime-session-id': '7ee2ed9f-b69b-4fc8-b0cc-7661d565a94c',
   'x-amzn-trace-id': 'Root=1-689187f4-7166e3444bf5d3271389aef2;Self=1-689187f4-7960c6cc6b4e66233d854361'},
  'RetryAttempts': 0},
 'runtimeSessionId': '7ee2ed9f-b69b-4fc8-b0cc-7661d565a94c',
 'traceId': 'Root=1-689187f4-7166e3444bf5d3271389aef2;Self=1-689187f4-7960c6cc6b4e66233d854361',
 'baggage': 'Self=1-689187f4-7960c6cc6b4e66233d854361,session.id=7ee2ed9f-b69b-4fc8-b0cc-7661d565a94c',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': [b'"The square 

In [19]:
invoke_response = agentcore_runtime.invoke({"prompt": "What time is it right now?"})
invoke_response

{'ResponseMetadata': {'RequestId': 'dde32448-8ea7-4c4c-9899-3e425dad7331',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 05 Aug 2025 04:26:37 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'dde32448-8ea7-4c4c-9899-3e425dad7331',
   'baggage': 'Self=1-689187fb-04f104da50462bfc4c58e6d3,session.id=7ee2ed9f-b69b-4fc8-b0cc-7661d565a94c',
   'x-amzn-bedrock-agentcore-runtime-session-id': '7ee2ed9f-b69b-4fc8-b0cc-7661d565a94c',
   'x-amzn-trace-id': 'Root=1-689187fb-5f339050243247ae650bb359;Self=1-689187fb-04f104da50462bfc4c58e6d3'},
  'RetryAttempts': 0},
 'runtimeSessionId': '7ee2ed9f-b69b-4fc8-b0cc-7661d565a94c',
 'traceId': 'Root=1-689187fb-5f339050243247ae650bb359;Self=1-689187fb-04f104da50462bfc4c58e6d3',
 'baggage': 'Self=1-689187fb-04f104da50462bfc4c58e6d3,session.id=7ee2ed9f-b69b-4fc8-b0cc-7661d565a94c',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': [b'"The current

### Processing invocation results

We can now process our invocation results to include it in an application

In [21]:
from IPython.display import Markdown, display
import json
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

The current time is 4:26 AM on August 5th, 2025.

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [22]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is 25 * 4?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                logger.info(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

The result of the calculation 25 * 4 is 100.

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [23]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

('686255965275.dkr.ecr.us-east-1.amazonaws.com/bedrock_agentcore-agentcore_semantic_kernel',
 'agentcore_semantic_kernel-znCs468wCI',
 'bedrock_agentcore-agentcore_semantic_kernel')

In [24]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_iam_role['Role']['RoleName']
)

# Congratulations!

You have successfully deployed a Semantic Kernel agent with Amazon Bedrock models to Amazon Bedrock AgentCore Runtime!

## What you learned:
- How to create a Semantic Kernel agent with custom plugins
- How to use function calling with Semantic Kernel and Amazon Bedrock
- How to deploy your agent to AgentCore Runtime
- How to invoke your deployed agent using both the toolkit and boto3

## Next steps:
- Try adding more sophisticated plugins to your agent
- Explore using different Amazon Bedrock models
- Learn about other AgentCore capabilities like Memory, Gateway, and Tools